In [30]:
import requests
import json
import pandas as pd
from datetime import datetime

webpage = requests.get("https://wusa.ca/events/")
jsonscript =str(webpage.content)
isolatedinformation=jsonscript.split('<script type="application/ld+json">')[1].split("</script>")[0][4:-4].encode("utf16", errors="surrogatepass").decode("utf16").encode().decode('unicode_escape')

json = json.loads(isolatedinformation.replace('&lt;p&gt;',"").replace("[&hellip;]&lt;/p&gt;\\\\n",""))

postsDf ={}
postscolumns = ['account','date','caption',"display_photo",'event_details']
postsDf = pd.DataFrame(columns = postscolumns)

print(json)

[{'@context': 'http://schema.org', '@type': 'Event', 'name': 'A Starry Night – Rx2024 Grad Ball', 'description': 'This year’s grad ball will be an unforgettable evening celebrating the graduating pharmacy class of Rx2024! ', 'image': 'https://8d3cdb9a.rocketcdn.me/wp-content/uploads/2024/05/August-23-2024-Delta-Hotel-Waterloo.jpg', 'url': 'https://wusa.ca/event/a-starry-night-rx2024-grad-ball/', 'offers': [{'@type': 'Offer', 'url': 'https://wusa.ca/event/a-starry-night-rx2024-grad-ball/', 'price': 55, 'category': 'primary', 'availability': 'InStock', 'priceCurrency': 'CAD', 'validFrom': '2024-05-17T00:00:00+00:00', 'validThrough': '2024-06-24T00:00:00+00:00'}, {'@type': 'Offer', 'url': 'https://wusa.ca/event/a-starry-night-rx2024-grad-ball/', 'price': 0, 'category': 'primary', 'availability': 'InStock', 'priceCurrency': 'CAD', 'validFrom': '2024-05-17T00:00:00+00:00', 'validThrough': '2024-06-24T00:00:00+00:00'}, {'@type': 'Offer', 'url': 'https://wusa.ca/event/a-starry-night-rx2024-gr

In [32]:
import time

time_now = int(time.time())
for event in json:
    if int(datetime.fromisoformat(event["startDate"]).timestamp()) >= time_now:
        try: 
            location = event["location"]["address"]["streetAddress"]
        except Exception as err:
            print(str(err))
            location = None
        
        new_row = pd.DataFrame({
                "account": ["WUSA"],
                "date": [time_now],
                "caption": str(event["description"])+" [For More Information, Click View Post] ",
                "display_photo": event["image"],
                "url": [event['url']],
                "likes": [0],
                "event_details": [{
                    "is_event": True,
                    "event_name": event["name"],
                    "event_description": str(event["description"])+" ... ",
                    "categories": ["Social"],
                    "start_time": int(datetime.fromisoformat(event["startDate"]).timestamp()),
                    "end_time": int(datetime.fromisoformat(event["endDate"]).timestamp()),
                    "location": location,
                }]
        })

        postsDf = pd.concat([postsDf, new_row])

print(postsDf)
postsDf.to_csv(path_or_buf="WUSA events")

'location'
'streetAddress'
'location'
'streetAddress'
'streetAddress'
  account        date                                            caption  \
0    WUSA  1724480921  The award-winning student orientation producti...   
0    WUSA  1724480921  📢 Hey WUSA Members! If this is your first time...   
0    WUSA  1724480921  CIBC x WUSA x UWaterloo presents Dillon Franci...   
0    WUSA  1724480921  Welcome Week 🍂 Get ready to FALL back into fun...   
0    WUSA  1724480921  Welcome Brunch Rise and shine, Warriors! WUSA’...   
0    WUSA  1724480921  Esports Gaming Lounge Game on, Warriors! Brace...   
0    WUSA  1724480921  Sex Toy Bingo 😍 A Welcome Week favourite is ba...   
0    WUSA  1724480921  WUSA\'s Carnival Step right up! The WUSA Carni...   
0    WUSA  1724480921  To give you a warm welcome back to campus, we ...   
0    WUSA  1724480921  WUSA\'s Carnival Step right up! The WUSA Carni...   
0    WUSA  1724480921  Retro Skate Night On Thursday, September 12th ...   
0    WUSA  1724480

In [28]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

In [33]:
from typing import List
from together import Together
import time

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

insertObjectIds = []

for index, row in postsDf.iterrows():
    info = f'"id": "{int(index)}"|* "account": "{row["account"]}"|* "date": "{row["date"]}"|* "caption": "{row["caption"]}"|*'
    embbededtext = ',\n'.join(x for x in info.replace('\n','\\n').split('|*')) 

    row_dict = {}
    for column in postsDf.columns.tolist():
        row_dict[column] = row[column]
    row_dict["embedded"] = generate_embedding([embbededtext], embedding_model_string)  
    result = collection.insert_one(row_dict)
    print(f"Inserted document ID: {result.inserted_id}")
    insertObjectIds.append(result.inserted_id)
    time.sleep(1)


Inserted document ID: 66c97dbafbd3dbb53a489523
Inserted document ID: 66c97dbcfbd3dbb53a489524
Inserted document ID: 66c97dbdfbd3dbb53a489525
Inserted document ID: 66c97dbffbd3dbb53a489526
Inserted document ID: 66c97dc0fbd3dbb53a489527
Inserted document ID: 66c97dc1fbd3dbb53a489528
Inserted document ID: 66c97dc2fbd3dbb53a489529
Inserted document ID: 66c97dc4fbd3dbb53a48952a
Inserted document ID: 66c97dc5fbd3dbb53a48952b
Inserted document ID: 66c97dc6fbd3dbb53a48952c
Inserted document ID: 66c97dc8fbd3dbb53a48952d
Inserted document ID: 66c97dc9fbd3dbb53a48952e
Inserted document ID: 66c97dcafbd3dbb53a48952f
Inserted document ID: 66c97dccfbd3dbb53a489530
Inserted document ID: 66c97dcdfbd3dbb53a489531
Inserted document ID: 66c97dcefbd3dbb53a489532
Inserted document ID: 66c97dcffbd3dbb53a489533
Inserted document ID: 66c97dd1fbd3dbb53a489534
Inserted document ID: 66c97dd2fbd3dbb53a489535
Inserted document ID: 66c97dd3fbd3dbb53a489536
Inserted document ID: 66c97dd5fbd3dbb53a489537
Inserted docu

In [29]:
from pymongo import MongoClient
from bson import ObjectId

# List of document IDs to be removed

for i in range(len(insertObjectIds)):
    insertObjectIds[i] = ObjectId(insertObjectIds[i])

# Remove all documents with these IDs
result = collection.delete_many({'_id': {'$in': insertObjectIds}})

# Print the number of documents deleted
print(f"Documents removed: {result.deleted_count}")


Documents removed: 22
